In [ ]:
!pip install pybullet
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install -q git+https://github.com/tensorflow/examples.git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [ ]:
import os
import cv2
import tensorflow as tf 
from tensorflow.keras import layers, models
import numpy as np 
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
import pybullet_envs
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

In [ ]:
np_seed = 654765645
tf_seed = 776644345
np.random.seed(np_seed)
tf.random.set_seed(tf_seed)

# check if GPU
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

root_dir = "drive/My Drive/"
base_dir = root_dir + 'CPCtesting'
os.makedirs(base_dir,exist_ok=True)

train_dir = base_dir + '/train'
os.makedirs(train_dir,exist_ok=True)

model_dir = base_dir + '/model'
os.makedirs(model_dir,exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# get data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [ ]:
class A2C(tf.keras.Model):
  def __init__(self):
    super(A2C,self).__init__()
    self.actor
    self.critic
  def optimizer(self):
    pass
  def loss(self):
    pass
  def call(self):
    pass

In [ ]:
class DataHandler:
    def __init__(self, batch_size, terms, predict_terms=1, image_size=64, color=False, rescale=True, aug=True, is_training=True, method='cpc'):
        self.batch_size = batch_size
        self.terms = terms
        self.predict_terms = predict_terms
        self.image_size = image_size
        self.color = color
        self.rescale = rescale
        self.aug = aug
        self.is_training = is_training
        self.method = method
        self.lena = cv2.imread(os.path.join(base_dir,'lena.jpg'))
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
        if self.is_training:
            self.x = x_train/255.0
            self.y = y_train
        else:
            self.x = x_test/255.0
            self.y = y_test
        self.idxs = []
        for i in range(10):
            y = y_train if self.is_training else y_test
            self.idxs.append(np.where(y == i)[0])
        self.n_samples = len(self.y)//terms if self.method == 'cpc' else len(self.y)
        self.shape = self.x.shape
        self.n_batches = self.n_samples//batch_size

    def __iter__(self):
        return self

    def __next__(self):
        return self.cpc_batch() if self.method == 'cpc' else self.benchmark_batch()

    def __len__(self):
        return self.n_batches

    def cpc_batch(self):
        img_labels = np.zeros((self.batch_size, self.terms + self.predict_terms))
        sentence_labels = np.ones((self.batch_size, 1)).astype('int32')
        for bi in range(self.batch_size):
            seed = np.random.randint(10)
            sentence = np.arange(seed, seed + self.terms + self.predict_terms) % 10
            if bi < self.batch_size//2:
                num = np.arange(10)
                predicted = sentence[-self.predict_terms:]
                for i, p in enumerate(predicted):
                    predicted[i] = np.random.choice(num[num != p], 1)
                sentence[-self.predict_terms:] = predicted % 10
                sentence_labels[bi, :] = 0
            img_labels[bi, :] = sentence
        images = self.get_samples(img_labels).reshape((self.batch_size, self.terms+self.predict_terms, self.image_size, self.image_size, 3))
        x_images = images[:, :-self.predict_terms, ...]
        y_images = images[:, -self.predict_terms:, ...]
        idx = np.random.choice(self.batch_size, self.batch_size, replace=False)
        return [x_images[idx], y_images[idx]], sentence_labels[idx]

    def get_samples(self, img_labels):
        idx = []
        for label in img_labels.flatten():
            idx.append(np.random.choice(self.idxs[int(label)], 1)[0])
        img_batch = self.x[idx, :, :]
        if self.aug:
            img_batch = self._aug_batch(img_batch)
        return img_batch

    def _aug_batch(self, img_batch):
        if self.image_size != 28:
            resized = []
            for i in range(img_batch.shape[0]):
                resized.append(cv2.resize(img_batch[i], (self.image_size, self.image_size)))
            img_batch = np.stack(resized)
        img_batch = img_batch.reshape((img_batch.shape[0], 1, self.image_size, self.image_size))
        img_batch = np.concatenate([img_batch, img_batch, img_batch], axis=1)

        if self.color:
            img_batch[img_batch >= 0.5] = 1
            img_batch[img_batch < 0.5] = 0
            for i in range(img_batch.shape[0]):
                x_c = np.random.randint(0, self.lena.shape[0] - self.image_size)
                y_c = np.random.randint(0, self.lena.shape[1] - self.image_size)
                img = self.lena[x_c:x_c+self.image_size, y_c:y_c+self.image_size]
                img = np.array(img).transpose((2, 0, 1))/255.0
                for j in range(3):
                    img[j, :, :] = (img[j, :, :] + np.random.uniform(0, 1))/2.0
                img[img_batch[i, :, :, :] == 1] = 1 - img[img_batch[i, :, :, :] == 1]
                img_batch[i, :, :, :] = img

        if self.rescale:
            img_batch = img_batch * 2 - 1
        img_batch = img_batch.transpose((0, 2, 3, 1))
        return img_batch

    def benchmark_batch(self):
        idx = np.random.choice(len(self.x), self.batch_size, replace=False)
        img_batch = self.x[idx]
        label_batch = self.y[idx]
        if self.aug:
            img_batch = self._aug_batch(img_batch)
        label_batch = label_batch.reshape((-1, 1))
        return img_batch, label_batch

In [ ]:
class CPCModel(tf.keras.Model):
  def __init__(self,code_size, predict_terms, terms=4, units=256, image_size=64, channels=3):
      super(CPCModel, self).__init__()
      self.code_size = code_size
      self.predict_terms = predict_terms
      self.terms = terms
      self.units = units
      self.image_size = image_size
      self.channels = channels

      self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
      self.bn1 = tf.keras.layers.BatchNormalization()
      self.lrelu1 = tf.keras.layers.LeakyReLU()
      self.conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
      self.bn2 = tf.keras.layers.BatchNormalization()
      self.lrelu2 = tf.keras.layers.LeakyReLU()
      self.conv3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
      self.bn3 = tf.keras.layers.BatchNormalization()
      self.lrelu3 = tf.keras.layers.LeakyReLU()
      self.conv4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')
      self.bn4 = tf.keras.layers.BatchNormalization()
      self.lrelu4 = tf.keras.layers.LeakyReLU()
      self.flatten = tf.keras.layers.Flatten()
      self.dense5 = tf.keras.layers.Dense(units=256, activation='linear')
      self.bn5 = tf.keras.layers.BatchNormalization()
      self.lrelu5 = tf.keras.layers.LeakyReLU()
      self.dense6 = tf.keras.layers.Dense(units=code_size, activation='linear', name='encoder_embedding')

      self.gru = tf.keras.layers.GRU(units, return_sequences=False, name='ar_context')
      self.linear = tf.keras.layers.Dense(predict_terms*code_size, activation='linear')    
  
  def encoding(self,x):
      x = self.conv1(x)
      x = self.bn1(x)
      x = self.lrelu1(x)
      x = self.conv2(x)
      x = self.bn2(x)
      x = self.lrelu2(x)
      x = self.conv3(x)
      x = self.bn3(x)
      x = self.lrelu3(x)
      x = self.conv4(x)
      x = self.bn4(x)
      x = self.lrelu4(x)
      x = self.flatten(x)
      x = self.dense5(x)
      x = self.bn5(x)
      x = self.lrelu5(x)
      z = self.dense6(x)
      return z
  
  def get_context(self, x):
      z = self.encoding(x)
      z = tf.reshape(z, [-1, self.terms, self.code_size])
      c = self.gru(z)
      return c

  def get_prediction(self, x):
      c = self.get_context(x)
      z_hats = self.linear(c)
      z_hat = tf.reshape(z_hats, [-1, self.predict_terms, self.code_size])
      return z_hat
  
  def optimizer(self):
    pass

  def loss(self,weights,biases,labels,inputs,num_samples,num_classes):
    
    loss = tf.nn.nce_loss(
    weights, biases, labels, inputs, num_sampled, num_classes, num_true=1,
    sampled_values=None, remove_accidental_hits=False, name='nce_loss')

    return loss


  def call(self,inputs):
    x_tm, x_tp = inputs
    x_tm = tf.reshape(x_tm, [-1, self.image_size, self.image_size, self.channels])
    x_tp = tf.reshape(x_tp, [-1, self.image_size, self.image_size, self.channels])
    z_hat = self.get_prediction(x_tm)
    z_tp = self.encoding(x_tp)
    z_tp = tf.reshape(z_tp, [-1, self.predict_terms, self.code_size])
    dot_prods = tf.reduce_mean(tf.reduce_mean(z_hat*z_tp, axis=-1), axis=-1, keepdims=True)
    probs = tf.sigmoid(dot_prods)
    return probs


  # def save(self):
  #       f1 = os.path.join(folder,'target_actor')
  #       f2 = os.path.join(folder, 'target_critic')
  #       f3 = os.path.join(folder, 'actor')
  #       f4 = os.path.join(folder, 'critic')
  #       self.target_actor.save(f1)
  #       self.target_critic.save(f2)
  #       self.actor.save(f3)
  #       self.critic.save(f4)


  # def load(self):
  #   pass

In [ ]:
#train loop
dh_train = DataHandler(64, 4, predict_terms=4, image_size=64, color=True, rescale=True, aug=True, is_training=True, method='cpc')
dh_test = DataHandler(64, 4, predict_terms=4, image_size=64, color=True, rescale=True, aug=True, is_training=False, method='cpc')
accuracy_metric_train = tf.keras.metrics.BinaryAccuracy()
loss_metric_train = tf.keras.metrics.BinaryCrossentropy()
accuracy_metric_test = tf.keras.metrics.BinaryAccuracy()
loss_metric_test = tf.keras.metrics.BinaryCrossentropy()
cpc = CPCModel(code_size=128, predict_terms=4, terms=4, units=256, image_size=64, channels=3)
optim = tf.keras.optimizers.Adam(1e-3)
cb = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=1/3, patience=2, min_lr=1e-4),
      tf.keras.callbacks.ModelCheckpoint('weights/weights.{epoch:02d}-{val_binary_accuracy:.2f}.cpkt',
                                          monitor='val_binary_accuracy', save_best_only=True, save_weights_only=True),
      tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=3),
      tf.keras.callbacks.TensorBoard()]
cpc.compile(optimizer=optim, loss='binary_crossentropy', metrics=['binary_accuracy'])
cpc.fit(x=dh_train, epochs=1, validation_data=dh_test, steps_per_epoch=60000//64, validation_steps=10000//64, callbacks=cb)


280/937 [=======>......................] - ETA: 19:42 - loss: 0.6941 - binary_accuracy: 0.4963

 https://datascience.stackexchange.com/questions/13216/intuitive-explanation-of-noise-contrastive-estimation-nce-loss(InfoNCE Loss )
<br>
Representation Learning with Contrastive Predictive Coding
<br>
https://github.com/gdao-research/cpc/blob/master/cpc/data_handler.py (CPC)
<br>
https://github.com/davidtellez/contrastive-predictive-coding/blob/master/train_model.py (CPC)